<a href="https://colab.research.google.com/github/metasir/AI-demo-notebooks/blob/master/AI_methods_in_a_nutshell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>[AI Methods and Techniques](#scrollTo=Fis92oy3kR-V&uniqifier=1)

>[GOFAI](#scrollTo=T8AjqfdNIDg9&uniqifier=1)

>>[A* Search](#scrollTo=pYbZpokpKzRi&uniqifier=1)

>>[Reinforcement Learning](#scrollTo=-oA779Zj9-AG&uniqifier=1)

>[Machine Learning](#scrollTo=Ky5Hux3VWjDz&uniqifier=1)

>[TensorFlow Playground](#scrollTo=PyDAqkyvmKeA&uniqifier=1)

>>[Exercises](#scrollTo=i9KGJsd_rf0K&uniqifier=1)



In [0]:
#@markdown [Imports]
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import random
import math
import time
import urllib.request
import ipywidgets as widgets
import tensorflow as tf
import tensorflow.keras as keras
from IPython.display import display
from IPython.display import clear_output
from google.colab import output

# AI Methods and Techniques

In this notebook we'll briefly look at various methods of dealing with hard problems and big data-sets. AI tends to fall into two main types: "Good Old-Fashioned AI" (or GOFAI), and "Machine Learning".

---

# GOFAI

GOFAI refers to the early attempts at creating AI, remaining dominant until the mid 1980's. GOFAI is also referred to as "Symbolic AI", because it largely uses symbolic representations of problems and tries to logically determine an optimal solution or policy of behavior. GOFAI is considered a failed approach to AI, because it doesn't solve real-world problems without substantial symbolic interpretation by the person employing it.

We'll cover a few examples of early GOFAI approaches to AI. For now we'll examine several **state-space search algorithms**, which try to find solutions for an environment with well-defined **states, transitions, and costs.**

<br>Start by generating a 2D world below. Our agent will start at START, and is trying to reach the END. Its movements are limited to the selected movement type, and the cost of each movement is shown as the cost to move into that square.

In [0]:
#@markdown [Generate world]

class World:
  '''
    Singleton class. The World class, once a world is generated, directly
    holds a map of the world. All functions are static, and can be called
    from the class.
  '''
  def __init__(self, **kwargs):
    self.__dict__.update(kwargs)
    World.world = self
    for key in kwargs:
      setattr(World, key, kwargs[key])

  # Get the cost of moving to a specific position.
  def get_cost(pos):
    return World.world_map[pos[0]][pos[1]]

  class Moves:
    ROOK_MOVES = [(0,1),(1,0),(0,-1),(-1,0)]
    KNIGHT_MOVES = [(a*i, b*j) for a,b in [(1,2),(2,1)] for i in (-1,1) for j in (-1,1)]
    BISHOP_MOVES = [(i,j) for i in (-1,1) for j in (-1,1)]
    KING_MOVES = ROOK_MOVES + BISHOP_MOVES

  # Get the valid *positions* to move to from the current position.
  def valid_moves(pos):
    ret = []
    for mov in World.moves:
      new_pos = (pos[0]+mov[0], pos[1]+mov[1])
      if new_pos[0] < 0 or new_pos[0] >= World.size or new_pos[1] < 0 or new_pos[1] >= World.size:
        continue
      ret.append(new_pos)
    return ret

  # Get the cost of a list of positions.
  def cost(path):
    tot = 0
    for tile in path:
      tot += World.get_cost(tile)
    return tot
  
  # Create a new singleton world.
  def generate_world(size, difficulty, moves):
    world_map = [[max(1, random.gauss(1 + (difficulty), difficulty)) for _ in range(size)] for __ in range(size)]
    world_map[0][0] = 1
    world_map[size-1][size-1] = 1
    World(size=size, world_map=world_map, moves=moves, start=(0,0), end=(size-1,size-1))

  # Draw the current world into a new plt figure, or existing figure/image if provided.
  def draw_world(ax=None, im=None):
    if not ax or not im:
      ax = plt.gca()
      im = ax.matshow(World.world_map, cmap=plt.gray())
      ax.figure.set_size_inches((0.8 * World.size,)*2)
      plt.close(ax.figure)
    else:
      im.set_data(World.world_map)
      im.set_clip_path(ax.patch)
      im.set_extent([-0.5, World.size-0.5, World.size-0.5, -0.5])
      im.autoscale()
      ax.figure.set_size_inches((0.8 * World.size,)*2)
      ax.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(nbins=World.size+1, steps=[1,2,5], integer=True, prune="both"))
      ax.yaxis.set_major_locator(matplotlib.ticker.MaxNLocator(nbins=World.size+1, steps=[1,2,5], integer=True, prune="both"))
    return ax, im
  
  # Print utility to get string representation of the current world.
  def raw_world():
    return "[[" + "],\n [".join(", ".join("{:.1f}".format(j) for j in i) for i in World.world_map) + "]]"

  # Set up interactive widgets to generate a new world.
  def setup_widgets():
    def gen_world(b):
      World.generate_world(size=size.value, difficulty=difficulty.value, moves=moves.value)
      with output.use_tags('world_image'):
        if hasattr(gen_world,"ax"):
          World.draw_world(gen_world.ax, gen_world.im)
        else:
          ax, im = World.draw_world()
          gen_world.ax = ax
          gen_world.im = im
        gen_world.ax.texts.clear()
        for i in range(World.size):
          for j in range(World.size):
            if (i,j) != World.start and (i,j) != World.end:
              gen_world.ax.annotate("{:.1f}".format(World.world_map[i][j]), xy=(j,i), ha="center", va="center", color="r")
        gen_world.ax.annotate("START", xy=World.start, ha="center", va="center", color="w")
        gen_world.ax.annotate("END", xy=World.end, ha="center", va="center", color="w")
        if not gen_world.initialized:
          gen_world.ax.figure.colorbar(mappable=gen_world.im, ax=gen_world.ax, fraction=0.042, pad=0.08, )
          plt.show(block=False)
          display(gen_world.ax.figure)
          gen_world.initialized = True
        else:
          output.clear(wait=True, output_tags=('world_image',))
          display(gen_world.ax.figure)

    gen_world.initialized = False

    moves = widgets.RadioButtons(options=[('Rook', World.Moves.ROOK_MOVES), 
                                          ('Knight', World.Moves.KNIGHT_MOVES), 
                                          ('Bishop', World.Moves.BISHOP_MOVES),
                                          ('King', World.Moves.KING_MOVES)], 
                                  description="Movement type:", value=World.Moves.ROOK_MOVES, disabled=False)
    difficulty = widgets.FloatSlider(1, min=1.0, max=10.0, step=0.1, description="Difficulty:")
    size = widgets.IntSlider(8, 4, 12, description="Size:")
    param_box = widgets.HBox(children=(size, difficulty, moves))
    world_output = widgets.Output()
    gen_map = widgets.Button(description="Generate World")
    gen_map.on_click(gen_world)
    display(param_box, gen_map, world_output)

    display(widgets.HTML(value="""<details><summary>Help:</summary><p>Generate a world for use in GOFAI examples. Each tile shows the cost of moving to that tile.<br>You can overwrite with a new random world using 'Generate World'.<ul><li>Size: set size of N by N board<li>Difficulty: increase variability of tile scores<li>Movement: set the available movement type for the board (like chess pieces)</ul></details>"""))
    
    gen_map.click()

World.setup_widgets()

## A* Search

<img src="https://artint.info/figures/ch03/searchsp.gif" width="400">

This search algorithm solves for the **optimal** path in a structured environment. A\* requires a notion of **path cost** and a **heuristic** to estimate the remaining distance to a goal. The heuristic _must_ provide the minimum possible distance from any point to the goal. Then A\* starts at the start and procedurally builds a **frontier** of paths to investigate and expand upon, until finally it comes upon the path reaching a goal node.

In [0]:
#@markdown [Show A* search]

class AStarPath:
  world_size = None
  
  def __init__(self, pos, hist, score=False):
    AStarPath.init_knight_table()
    self.pos = pos
    self.hist = hist
    self.g = World.cost(self)
    self.h = AStarPath.heuristic(pos)
    self.score = score or self.g + self.h
  
  def __iter__(self):
    return iter(self.hist + [self.pos])
  
  def __getitem__(self, item):
    return self.pos[item]
  
  def __repr__(self):
    return str([self.hist + [self.pos], self.score])

  def init_knight_table():
    if World.size != AStarPath.world_size:
      AStarPath.world_size = World.size
    else:
      return
    AStarPath.KNIGHT_TABLE = {World.end: 0}
    for i in range(12):
      starts = [key for key in AStarPath.KNIGHT_TABLE.keys() if AStarPath.KNIGHT_TABLE[key] == i-1]
      for start in starts:
        moves = [mov for mov in World.valid_moves(start) if mov not in AStarPath.KNIGHT_TABLE]
        for mov in moves:
          AStarPath.KNIGHT_TABLE[mov] = i
    
  def heuristic(pos):
    move_type = World.moves
    if move_type == World.Moves.ROOK_MOVES:
      return 2*World.size - 2 - sum(pos)
    elif move_type == World.Moves.BISHOP_MOVES or move_type == World.Moves.KING_MOVES:
      return World.size-1 - min(pos)
    elif move_type == World.Moves.KNIGHT_MOVES:
      return AStarPath.KNIGHT_TABLE[pos]

  def new_paths(self):
    moves = World.valid_moves(self.pos)
    return [AStarPath(mov, self.hist + [self.pos], self.score + World.get_cost(mov) - self.h + AStarPath.heuristic(mov)) for mov in moves]

def astar_solve_world():
  path_stack = [AStarPath(World.start, [])]
  visited = []
  path_count = 0
  while path_stack:
    path = path_stack.pop(0)
    if path.pos in visited:
      continue
    if path.pos == World.end:
      plt.pause(1.0 / display_frequency.value)
      return path
    path_count += 1
    if path_count % display_interval.value == 0:
      display_path(path)
      plt.pause(1.0 / display_frequency.value)
    visited.append(path.pos)
    new_paths = path.new_paths()
    for npath in new_paths:
      if npath.pos in visited:
        continue
      i = next((j for j in range(len(path_stack)) if path_stack[j].score > npath.score), len(path_stack))
      path_stack.insert(i, npath)
  raise "No solution found"

# This function maintains attributes .ax and .im to track its current plt figure.
# This solves widget problems.
def display_path(path, solved=False):
  astar.ax.texts.clear()
  astar.ax.annotate("START", xy=World.start, ha="center", va="center", color="w")
  i = 1
  for pos in path.hist[1:]:
    astar.ax.annotate(str(i), xy=pos[::-1], ha="center", va="center", color="r")
    i += 1
  if solved == True:
    astar.ax.annotate("END", xy=World.end, ha="center", va="center", color="w")
  output.clear(wait=True, output_tags='astar')
  display(astar.ax.figure)
  if solved:
    display(path)
  
def astar(b):
  output.clear(wait=True, output_tags='astar')
  with output.use_tags('astar'):
    astar.ax, astar.im = World.draw_world()
    solution_path = astar_solve_world()
    display_path(solution_path, solved=True)

astar_button = widgets.Button(description="Run AStar")
display_interval = widgets.FloatLogSlider(value=4, base=2, min=1, max=6, step=1)
display_frequency = widgets.FloatSlider(1, min=0.5, max=2)
astar_button.on_click(astar)
display(widgets.HBox((astar_button, widgets.HTML("<span style='display:inline-block;width:30px'></span>Refresh interval:"), display_interval, widgets.HTML("<span style='display:inline-block;width:30px'></span>Refresh rate:"), display_frequency)))
display(widgets.HTML(value="""<details><summary>Help:</summary><p>Run AStar to find the best path for the currently generated World.<li>Refresh interval: refresh the map with the current best path every N steps<li>Refresh rate: Time between refreshes</ul></details>"""))

with output.use_tags('astar'):
  output.clear(wait=True, output_tags='astar')
  astar.ax, astar.im = World.draw_world()
  display(astar.ax.figure)


## Reinforcement Learning

Reinforcement learning falls in between GOFAI and machine learning. The reinforcement learning approach deliberately maims the agent by refusing to tell it the rules of the game. Instead, the agent is allowed to attempt an action, and the world will report to the agent the result of its action. The agent will slowly figure out how well an action worked from each state, and should eventually have a **policy** dictating the value of each action at each state. Then the agent should simply choose the best action available at each state.

We'll specifically be using q-learning.

In [0]:
#@markdown [Show reinforcement learning]
class QPolicy:
  def __init__(self, explore_rate=0.2, learn_rate=0.25, discount_factor=0.98, reward=20, goal_bias=0.10):
    self.q_map = {pos: {mov: 0 for mov in World.valid_moves(pos)} 
             for pos in [(i,j) for i in range(World.size) for j in range(World.size)]}
    self.explore_rate = explore_rate
    self.learn_rate = learn_rate
    self.discount_factor=discount_factor
    self.reward = reward
    self.goal_bias=goal_bias
    self._knight_init = False

  def heuristic(self, pos):
    def init_knight_table(self):
      self.KNIGHT_TABLE = {World.end: 0}
      for i in range(min(4, World.size) + 5):
        starts = [key for key in self.KNIGHT_TABLE.keys() if self.KNIGHT_TABLE[key] == i-1]
        for start in starts:
          moves = [mov for mov in World.valid_moves(start) if mov not in self.KNIGHT_TABLE]
          for mov in moves:
            self.KNIGHT_TABLE[mov] = i
    
    move_type = World.moves
    if move_type == World.Moves.ROOK_MOVES:
      return 2*World.size - 2 - sum(pos)
    elif move_type == World.Moves.BISHOP_MOVES or move_type == World.Moves.KING_MOVES:
      return World.size-1 - min(pos)
    elif move_type == World.Moves.KNIGHT_MOVES:
      if not self._knight_init:
        init_knight_table(self)
        self._knight_init=True
      return self.KNIGHT_TABLE[pos]

  def get_q_value(self, pos, action):
    return self.q_map[pos].get(action, -math.inf)

  def best_action(self, pos):
    best_choices = [act for act in self.q_map[pos] if math.isclose(self.get_q_value(pos, act), max(self.get_q_value(pos, act_) for act_ in self.q_map[pos]))]
    if len(best_choices) == 1:
      return best_choices[0]
    else:
      return None
  
  def best_path(self, start=(0,0)):
    pos = start
    path = []
    while pos != World.end:
      if pos in path:
        return []
      path.append(pos)
      pos = self.exploit(pos)
    return path + [World.end]

  def adjust_q_value(self, pos, action, adjustment):
    self.q_map[pos][action] += adjustment

  def exploit(self, pos):
    best_choices = [act for act in self.q_map[pos] if math.isclose(self.get_q_value(pos, act), max(self.get_q_value(pos, act_) for act_ in self.q_map[pos]))]
    if len(best_choices) == 1:
      return best_choices[0]
    else:
      return best_choices[np.random.randint(low=0, high=len(best_choices)-1)]

  def explore(self, pos):
    options = self.q_map[pos]
    return list(options)[np.random.randint(low=0, high=len(options))]
  
  def episode(self):
    path = []
    pos = tuple(np.random.randint(low=0, high=World.size-1, size=2))
    while pos != World.end:
      path.append(pos)
      do_exploit = (np.random.random() > self.explore_rate)
      next_pos = self.exploit(pos) if do_exploit else self.explore(pos)
      reward = self.reward if (next_pos == World.end) else -World.get_cost(next_pos) + self.goal_bias * (self.heuristic(pos) - self.heuristic(next_pos))
      self.adjust_q_value(pos, next_pos, self.learn_rate * (reward + self.discount_factor * self.get_q_value(next_pos, max(self.q_map[next_pos], key=lambda x: self.get_q_value(next_pos, x))) - self.get_q_value(pos, next_pos)))
      pos = next_pos
    return path
  
  def display(self, ax=None, im=None):
    if not ax or not im:
      ax, im = World.draw_world()
    for i in range(World.size):
      for j in range(World.size):
        if (i,j) == World.end:
          ax.annotate("END", xy=World.end, ha="center", va="center", color="w")
          continue
        best = self.best_action((i,j))
        if not best:
          ax.annotate("?", xy=(j,i), ha="center", va="center", color="r")
        else:
          ax.annotate("", xy=best[::-1], xytext=(j,i), 
                      arrowprops={"color": "red", "arrowstyle": "->", "shrinkB": 10}, ha="center", va="center", color="r")
    output.clear(wait=True, output_tags='q_learn')
    display(ax.figure)

widget_style = {"description_width": "initial"}
q_widgets = {"explore_rate": widgets.FloatSlider(value=.2, min=0.05, max=0.5, step=0.05, description="Explore rate", style=widget_style),
             "learn_rate": widgets.FloatSlider(value=0.25, min=0.05, max=0.5, step=0.05, description="Learn rate", style=widget_style),
             "discount_factor": widgets.FloatSlider(value=0.98, min=0.8, max=1.0, step=0.01, description="Discount factor", style=widget_style),
             "reward": widgets.IntSlider(value=20, min=5, max=200, step=5, description="Reward", style=widget_style),
             "goal_bias": widgets.FloatSlider(value=0.10, min=0.0, max=0.20, step=0.01, description="Goal bias", style=widget_style),
             "learn_box": widgets.HBox(children=(
                widgets.Button(description="Run Q-Learning"),
                widgets.IntText(value=500, step=100, description="# of episodes:", style=widget_style),
                widgets.Checkbox(value=False, description="Show progress? (SLOW)", style=widget_style))),
             "episode_box": widgets.HBox(children=(
                widgets.Button(description="Run single episode"),
                widgets.IntSlider(value=1, min=1, max=100, step=1, description="Episode #:", style=widget_style))),
             }

def q_learn(b):
  q = QPolicy(**{a: q_widgets.get(a).value for a in ["explore_rate", "learn_rate", "discount_factor", "reward", "goal_bias"]})
  num_episodes = q_widgets.get("learn_box").children[1].value
  with output.use_tags('q_learn'):
    for i in range(num_episodes):
      q.episode()
      if q_widgets["learn_box"].children[2].value and (i+1) % (math.floor(num_episodes / 10.0)) == 0:
        q.display()
        time.sleep(1)
    q.display()
    best_path = q.best_path()
    if best_path:
      print(best_path, World.cost(best_path))
    else:
      print("[!] Policy contains inescapable recursive loop.")

def q_episode(b):
  q = QPolicy(**{a: q_widgets.get(a).value for a in ["explore_rate", "learn_rate", "discount_factor", "reward", "goal_bias"]})
  for _ in range(q_widgets["episode_box"].children[1].value):
    q.episode()
  path = q.episode()
  with output.use_tags('q_learn'):
    output.clear(wait=True, output_tags='q_learn')
    ax, im = World.draw_world()
    ax.annotate("", xy=path[1][::-1], xytext=path[0][::-1], arrowprops={"color": "red", "arrowstyle": "->", "shrinkB": 10}, ha="center", va="center")
    for i in range(1,len(path) - 1):
      ax.annotate("", xy=path[i+1][::-1], xytext=path[i][::-1], arrowprops={"color": "#{0:02x}{1:02x}{2:02x}".format(5*i % 256, (50 + 3*i) % 256, (80 + i) % 256), "arrowstyle": "->", "shrinkB": 10}, ha="center", va="center")
    ax.annotate("", xy=World.end, xytext=path[-1][::-1], arrowprops={"color": "red", "arrowstyle": "->", "shrinkB": 10}, ha="center", va="center")
    ax.annotate("S", xy=path[0][::-1], va="center", ha="center", color="#ff33ff")
    output.clear(wait=True, output_tags='q_learn')
    display(ax.figure)

q_widgets["learn_box"].children[0].on_click(q_learn)
q_widgets["episode_box"].children[0].on_click(q_episode)

display(*q_widgets.values())
display(widgets.HTML(value="""\
<details><summary>Help:</summary>
<p>Create a policy for the current world by using <u>Run Q-Learning</u>.<br>
View what the agent does in a single episode by using <u>Run single episode</u>, using the <u>Episode #</u> slider to see the Nth episode.
<ul>
  <li>Explore rate: set the rate at which the agent chooses a random direction instead of the current best.
  <li>Learn rate: set how quickly the agent gains knowledge of the results of each action.
  <li>Discount factor: set the discount factor for each successive tile. The lower this is, the more the agent is punished for taking a long time to get to the goal.
  <li>Reward: set the reward level for reaching the goal tile. This reward is what teaches the agent that moving towards the goal is a good thing. This generally should scale with the board difficulty and size.
  <li>Goal bias: this bias gives the agent a slight immediate reward for heading in the direction of the goal. This speeds up the initial episodes by preventing the agent from going in circles at random.
  <li># of Episodes: the agent is dropped into the world and explores or progresses towards the goal this many times. Note that for a board with 100 tiles, 100 episodes would mean several tiles are likely not to ever be started on.
</ul></details>\
"""))

with output.use_tags('q_learn'):
  ax, im = World.draw_world()
  display(ax.figure)


---
# Machine Learning


<img src="https://cdn.app.compendium.com/uploads/user/e7c690e8-6ff9-102a-ac6d-e4aebca50425/f0499405-1197-4b43-b7c5-40548eeb9f34/Image/5f0b569126b3f2da0538b3ce34bd89d2/difference_between_ai__machine_learning_and_deep_learning.png" width="800">

Machine learning followed the decline of GOFAI by relying on new *data driven* methodologies. Machine learning is often wrongly conflated with "artificial intelligence", "deep learning", "neural networks". Here's how to read those terms:
- **Machine learning**: the methodology in artificial intelligence which infers patterns and rules from natural data, but chiefly not from pre-determined rules. These algorithms create a model that can be applied on new data-sets.
- **Deep Learning**: the class of machine learning algorithms which use multiple layers, each tuned to solving a sub-problem or learning specific problems, to extract higher level features from an input.
- **Neural Networks**: an application of deep learning which connects layers of nodes in a network, where the entire network is trained on a problem together. Each layer of nodes takes a previous layer of nodes in the network as input, and typically each node provides a single output (usually between -1 and 1).

<br><br><br>
<img src="https://miro.medium.com/max/775/1*Qn4eJPhkvrEQ62CtmydLZw.png" width="600">

Further, machine learning algorithms are split into two types:
- **Regression**: these algorithms provide continuous output, usually consisting of a set of outputs totaling 1. These outputs usually represent the confidence in an assessment, e.g. 70% (or 0.7) chance of rain.
- **Classification**: these algorithms apply discrete labels, determining what type of pattern is identified in an input, e.g. "This is a hot dog / this is not a hot dog." Often, a classification model can be derived from a regression model by simply classifying a sample according to a highest scoring regression model.

<br><br><br>
<img src="https://www.researchgate.net/publication/329533120/figure/fig1/AS:702267594399761@1544445050584/Supervised-learning-and-unsupervised-learning-Supervised-learning-uses-annotation.png">

*Further*, these algorithms are trained in one of two ways:
- **Supervised**: data provided to the agent is labeled, so that the model associates patterns to specific labels.
- **Unsupervised**: data is *not* labeled. The agent assesses a data-set and begins to group similar samples, identifying patterns. This provides a model which assesses new data to be 'like' previously identified patterns or groups of data.
- **Reincorcement learning**: data is *created* by the agent. This is considered machine learning because it is driven by data, but counts as GOFAI because the data is artificial and can operate in purely theoretical domains.

In [0]:
#@markdown [Import MNIST dataset]
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
(x_train, x_test) = (x_train / 255.0, x_test / 255.0)

In [0]:
#@markdown [Create plot utilities]
def plot_image(predictions_array, true_label, img):
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predictions_array = predictions_array / sum(predictions_array)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = '#D0D0D0'
  else:
    color = '#D08080'

  plt.xlabel("{} {:2.0f}% ({})".format(predicted_label,
                                100*np.max(predictions_array),
                                true_label),
                                color=color)

def plot_value_array(predictions_array, true_label):
  predictions_array = predictions_array / sum(predictions_array)
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777", )
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

Let's examine the MNIST dataset. This dataset has a large number of 28x28 sized grayscale images of hand-written digits, 0-9. 

In [0]:
#@markdown [Show image examples]
plt.figure(figsize=(10,2))
for i in range(5):
    plt.subplot(1,5,i+1)
    plt.imshow(x_test[i+5], cmap=plt.cm.binary)
    plt.xticks([])
    plt.yticks([])
    plt.xlabel("{}".format(y_test[i+5]), color="#D0D0D0")

Below, we'll create, compile, and train 4 models using Keras with TensorFlow:
- Best of 10 1-node linear regression models
- Best of 10 1-node logistic regression models
- Neural network classification with 140-node linear hidden layer
- Neural network classification with 140-node logistic hidden layer

In [0]:
#@markdown [Create models]
linear_model = keras.Sequential([
                                 keras.layers.Flatten(input_shape=(28,28)), 
                                 keras.layers.Dense(10, activation='relu', bias_initializer=keras.initializers.RandomNormal()),
                                 keras.layers.Lambda(lambda x: x**2)
                                 ])

logistic_model = keras.Sequential([
                                   keras.layers.Flatten(input_shape=(28,28)), 
                                   keras.layers.Dense(10, activation='sigmoid', bias_initializer=keras.initializers.RandomNormal()),
                                   keras.layers.Lambda(lambda x: x**2)
                                   ])

neural_linear_model = keras.Sequential([
                                        keras.layers.Flatten(input_shape=(28,28)), 
                                        keras.layers.Dense(140, activation='relu', bias_initializer=keras.initializers.RandomNormal()), 
                                        keras.layers.Dense(10, activation='softmax')
                                        ])

neural_logistic_model = keras.Sequential([
                                          keras.layers.Flatten(input_shape=(28,28)), 
                                          keras.layers.Dense(140, activation='sigmoid', bias_initializer=keras.initializers.RandomNormal()), 
                                          keras.layers.Dense(10, activation='softmax')
                                          ])

In [0]:
#@markdown [Compile models]
linear_model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.0001), 
                     loss=keras.losses.SparseCategoricalCrossentropy(), 
                     metrics=['accuracy'])

logistic_model.compile(optimizer=keras.optimizers.SGD(), 
                       loss=keras.losses.SparseCategoricalCrossentropy(), 
                       metrics=['accuracy'])

neural_linear_model.compile(optimizer=keras.optimizers.SGD(), 
                            loss=keras.losses.SparseCategoricalCrossentropy(), 
                            metrics=['accuracy'])

neural_logistic_model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.001), 
                              loss=keras.losses.SparseCategoricalCrossentropy(), 
                              metrics=['accuracy'])

In [0]:
#@markdown [Train models]
num_epochs = 5 #@param {"type": "slider", "min": 1, "max": 25}
linear_model.fit(x_train, y_train, epochs=num_epochs)
logistic_model.fit(x_train, y_train, epochs=num_epochs)
neural_linear_model.fit(x_train, y_train, epochs=num_epochs)
neural_logistic_model.fit(x_train, y_train, epochs=num_epochs)

In [0]:
#@markdown [Test samples]
num_samples = 20 #@param {'type': 'slider', 'min': 1, 'max': 100}
sample_indices = np.random.choice(x_test.shape[0], size=num_samples, replace=False)
x_sample = np.array([x_test[i] for i in sample_indices])
y_sample = np.array([y_test[i] for i in sample_indices])
lin_predicts = linear_model.predict(x_sample)
log_predicts = logistic_model.predict(x_sample)
neural_lin_predicts = neural_linear_model.predict(x_sample)
neural_log_predicts = neural_logistic_model.predict(x_sample)

In [0]:
#@title Linear model predictions
sampnum_5 = math.ceil(num_samples / 5.0)
plt.figure(figsize=(20,2*sampnum_5))
for i in range(num_samples):
    plt.subplot(sampnum_5,10,2*i+1)
    plot_image(lin_predicts[i], y_sample[i], x_sample[i])
    plt.subplot(sampnum_5,10,2*i+2)
    plot_value_array(lin_predicts[i], y_sample[i])
    
plt.show()

In [0]:
#@title Logistic model predictions
sampnum_5 = math.ceil(num_samples / 5.0)
plt.figure(figsize=(20,2*sampnum_5))
for i in range(num_samples):
    plt.subplot(sampnum_5,10,2*i+1)
    plot_image(log_predicts[i], y_sample[i], x_sample[i])
    plt.subplot(sampnum_5,10,2*i+2)
    plot_value_array(log_predicts[i], y_sample[i])
    
plt.show()

In [0]:
#@title Neural linear model predictions
sampnum_5 = math.ceil(num_samples / 5.0)
plt.figure(figsize=(20,2*sampnum_5))
for i in range(num_samples):
    plt.subplot(sampnum_5,10,2*i+1)
    plot_image(neural_lin_predicts[i], y_sample[i], x_sample[i])
    plt.subplot(sampnum_5,10,2*i+2)
    plot_value_array(neural_lin_predicts[i], y_sample[i])
    
plt.show()

In [0]:
#@title Neural logistic model predictions
sampnum_5 = math.ceil(num_samples / 5.0)
plt.figure(figsize=(20,2*sampnum_5))
for i in range(num_samples):
    plt.subplot(sampnum_5,10,2*i+1)
    plot_image(neural_log_predicts[i], y_sample[i], x_sample[i])
    plt.subplot(sampnum_5,10,2*i+2)
    plot_value_array(neural_log_predicts[i], y_sample[i])
    
plt.show()

# TensorFlow Playground

Experiment in the playground to learn more about how neural networks work, or [go to tensorflow's website to view more.](https://playground.tensorflow.org)

In [0]:
#@markdown [Load TensorFlow Playground]
%%html
<iframe width="1400" height="1000" name="tf_playground" src="https://playground.tensorflow.org#activation=sigmoid"></iframe>

##Exercises
Run each exercise to open its specialized playground.

In [0]:
#@markdown ####What biases predict this dataset well with no hidden layers?
#@markdown <details><summary>Hint:</summary><p>What line would you use? What's the <i>slope</i> of that line?<p>(Pay careful attention to how x and y are oriented on the graph.)</details>

%%html
<iframe width="1400" height="700" src="https://playground.tensorflow.org/#activation=linear&batchSize=10&dataset=gauss&regDataset=reg-plane&learningRate=0.0001&regularizationRate=0&noise=0&networkShape=&seed=0.55382&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false&showTestData_hide=true&learningRate_hide=true&regularizationRate_hide=true&percTrainData_hide=true&numHiddenLayers_hide=true&discretize_hide=true&activation_hide=true&problem_hide=true&noise_hide=true&regularization_hide=true&dataset_hide=true&batchSize_hide=true&playButton_hide=false"></iframe>

In [0]:
#@markdown ####With 1 hidden layer, how many neurons are required to get a decent model?
#@markdown <details><summary>Hint:</summary><p>If you were drawing lines to capture the inner circle, how many lines would it take to enclose it?</details>

%%html
<iframe width="1400" height="700" src="https://playground.tensorflow.org/#activation=sigmoid&regularization=L2&batchSize=10&dataset=circle&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=0&networkShape=1&seed=0.84062&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&showTestData_hide=true&learningRate_hide=true&regularizationRate_hide=true&percTrainData_hide=true&numHiddenLayers_hide=true&discretize_hide=true&activation_hide=true&problem_hide=true&noise_hide=true&regularization_hide=true&dataset_hide=true&batchSize_hide=true&playButton_hide=false"></iframe>

In [0]:
#@markdown ####With no hidden layers, which 2 inputs can produce a decent model?
#@markdown <details><summary>Hint:</summary><p>Do you remember the equation for a circle?</details>

%%html
<iframe width="1400" height="700" src="https://playground.tensorflow.org/#activation=sigmoid&regularization=L2&batchSize=10&dataset=circle&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=0&networkShape=&seed=0.84062&showTestData=false&discretize=false&percTrainData=50&x=false&y=false&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false&showTestData_hide=true&learningRate_hide=true&regularizationRate_hide=true&percTrainData_hide=true&numHiddenLayers_hide=true&discretize_hide=true&activation_hide=true&problem_hide=true&noise_hide=true&regularization_hide=true&dataset_hide=true&batchSize_hide=true&playButton_hide=false"></iframe>

In [0]:
#@markdown ####Let this model run. Pay attention to the test loss and training loss. Is the model improving? Why or why not?
#@markdown <details><summary>Hint:</summary><p>The data shown in the picture is the <i>training data</i>. How well does it represent the real data this model would be tested on?<p>(Also take note of the noise level for this dataset.)</details>

%%html
<iframe width="1400" height="700" src="https://playground.tensorflow.org/#activation=tanh&batchSize=10&dataset=circle&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=30&networkShape=6,4,3&seed=0.94343&showTestData=false&discretize=false&percTrainData=70&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false&dataset_hide=true&regularization_hide=true&batchSize_hide=true&problem_hide=true&activation_hide=true&regularizationRate_hide=true&percTrainData_hide=false&numHiddenLayers_hide=true&learningRate_hide=true"></iframe>

In [0]:
#@markdown ####How much wood would a woodchuck chuck if a woodchuck could chuck wood?
#@markdown <details><summary>Hint:</summary><p>A woodchuck would chuck as much wood as a woodchuck could chuck if a woodchuck could chuck wood.</details>
#@markdown <p>(No exercise - this is just an interesting neural network.)
%%html
<iframe width="1400" height="700" src="https://playground.tensorflow.org/#activation=relu&batchSize=10&dataset=spiral&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=0&networkShape=8,8,5&seed=0.53586&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&showTestData_hide=true&activation_hide=true&problem_hide=true&noise_hide=true&discretize_hide=true&regularization_hide=true&dataset_hide=true&batchSize_hide=true&learningRate_hide=true&regularizationRate_hide=true&percTrainData_hide=true&numHiddenLayers_hide=true"></iframe>